# Can we detect climate misinformation by analysing a single word or phrase?

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import sys

sys.path.insert(0, '../text_preprocessing/')

In [2]:
from preprocessing import *
from embeddings import *

/usr/local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
path = '../labelled_data/labelled_data_14032021.csv'

print("Importing data...")
df = import_data(path)

print("\nDropping na values..")
df = na_values(df)

print("\nEncoding classes..")
df, label_columns = class_encoding(df)

# print("\nFiltering long doocuments and exploring length..")
# documents_len_exploration_and_filter(df,max_len=1500)

print("\nStarting text preprocessing..")
clean_text = preprocessing(df)

df['clean_text'] = clean_text

print(len(df))
df.head()

../text_preprocessing/preprocessing.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['human_label'] = encoded_classes
../text_preprocessing/preprocessing.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['human_binary_label'] = df_encoded['human_label'].apply(lambda label: 1 if label > 0 else 0)


Importing data...
   label                                               text
0    118  Buenos Aires Playa is an urban beach on the Rí...
1    123  Contributed by Robert Lyman  © 2017The United ...
2    119  Contributed by Robert Lyman  © 2017The United ...
3    119  Share this...FacebookTwitterState parliamentar...
4    122  [CHECK OUT OUR PLANS](https://thebfd.co.nz/sub...
Size of dataframe
Index(['label', 'text'], dtype='object')
(778, 2)

Dropping na values..
NaN values per column
label     0
text     62
dtype: int64

Encoding classes..
118 is encoded to [0]
119 is encoded to [1]
120 is encoded to [2]

Starting text preprocessing..


## Wordclouds for each Class

In [ ]:
# Same code can be used to generate wordclouds of other categories.
def plot_wordcloud(label):

    plt.figure(figsize=(35,20))

#     text = df[df.human_label == label].text.fillna("").values
    full_text = ""
    for doc in df[df.human_label == label].clean_text.to_list():
        for text in doc:
            full_text = "{0} {1}".format(full_text,text).strip()
    
    cloud_text = WordCloud(
                              stopwords=STOPWORDS,
                              background_color='black',
                              collocations=False,
                              width=2000,
                              height=1000
                             ).generate(full_text) #" ".join(text))
    plt.axis('off')
    plt.title('Word cloud for "{0}"\n'.format(label),fontsize=40)
    plt.imshow(cloud_text)
    plt.show()

for col in df.human_label.unique():
#     print(col)
    plot_wordcloud(col)
    print("\n\n#########\n\n")

## Most frequent Keywords

In [ ]:
top_n = []
n = 500

for col in df.human_label.unique():
    print(col)
    words = []
    for doc in df.loc[df.human_label == col, 'clean_text'].to_list():
        words += doc

    d = {
        "label": col,
        "top_{0}".format(n): pd.Series(words).value_counts()[:n].index
    }
    top_n.append(d)

top_n

In [ ]:
def word_analysis(word_list, min_length=4, min_freq=0.1, max_freq_other=0.1):
    i = 0

    # min_length = 4
    # min_freq = 0.1
    # max_freq_other = 0.1

    for word in sorted(word_list):
        if len(word) < min_length:
            continue

    #     freq = df.loc[df.text.fillna("").str.contains(word, regex=False, case=False).fillna(False), "human_label"].value_counts() / df.human_label.value_counts()
        mask = df.clean_text.apply(lambda x: word in x)
        freq = df.loc[mask, "human_label"].value_counts() / df.human_label.value_counts()
        if (freq[0] < min_freq) or (freq[1] > max_freq_other) and (freq[2] > max_freq_other):
            continue

        i += 1
        print("\n\n##### {0}\n".format(word))
        print(freq)

    print("\n\n##### {0} words selected for analysis".format(i))

In [ ]:
word_analysis(top_n[0]["top_{0}".format(n)], min_freq=0.3, max_freq_other=0.1)

## Most frequent Keywords (misinfo focus)

In [ ]:
top_misinfo_only = []

for val in top_n[0]["top_{0}".format(n)]:
    if (val in top_n[1]["top_{0}".format(n)]) or (val in top_n[2]["top_{0}".format(n)]):
        continue
    top_misinfo_only.append(val)

len(top_misinfo_only), top_misinfo_only

In [ ]:
# i = 0

# min_length = 4
# min_freq = 0.1
# max_freq_other = 0.1

# for word in sorted(top_misinfo_only):
#     if len(word) < min_length:
#         continue
    
# #     freq = df.loc[df.text.fillna("").str.contains(word, regex=False, case=False).fillna(False), "human_label"].value_counts() / df.human_label.value_counts()
#     mask = df.clean_text.apply(lambda x: word in x)
#     freq = df.loc[mask, "human_label"].value_counts() / df.human_label.value_counts()
#     if (freq[0] < min_freq) or (freq[1] > max_freq_other) and (freq[2] > max_freq_other):
#         continue
        
#     i += 1
#     print("\n\n##### {0}\n".format(word))
#     print(freq)
    
# print("\n\n##### {0} words selected for analysis".format(i))

word_analysis(top_misinfo_only)

## Curated Keywords from Notion

In [ ]:
notion_words_misinfo = ["climatecult", "climatehysteria", "globalcooling", "climatescam", "climatehoax", "climatechangehoax",
                        "greenwashing"]

notion_words_climate = ["TheTimeIsNow", "weneedexperts", "ecology", "climatechange", "HowtoSavethePlanet", "VoiceForThePlanet",
                        "FightForYourWorld", "connect2earth", "globalwarming"]

notion_words_notrel = ["coronavirus"]

notion_keywords = [
    {"label": "Climate Misinformation", "words": notion_words_misinfo},
    {"label": "Climate Related", "words": notion_words_climate},
    {"label": "Not Climate Related", "words": notion_words_notrel},
]

for k in notion_keywords:
    print("\n-------------------------------\n{0}\n{1}".format(k["label"], k["words"]))

# df.loc[df.text.fillna("").str.contains("({0})".format("|".join(notion_words_misinfo), case=False)), "human_label"].value_counts() / df.human_label.value_counts()

#     for word in k["words"]:
# #         freq = df.loc[df.text.fillna("").str.contains(word, regex=False, case=False).fillna(False), "human_label"].value_counts() / df.human_label.value_counts()
#         mask = df.clean_text.apply(lambda x: word in x)
#         freq = df.loc[mask, "human_label"].value_counts() / df.human_label.value_counts()
#         print("\n\n##### {0}\n".format(word))
#         print(freq)

    word_analysis(k["words"], min_freq=0, max_freq_other=1)